<a href="https://colab.research.google.com/github/junaid537/pytorch/blob/main/%E2%80%9CHW2_Junaid_Khalidi_Py_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import psutil
total_memory = psutil.virtual_memory().total / 1e9
print(f"Total RAM: {total_memory:.2f} GB")

Total RAM: 13.61 GB


In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.6 gigabytes of available RAM

Not using a high-RAM runtime


In [ ]:
pip install --upgrade --force-reinstall nltk


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 792.7/792.7 kB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 5.7 MB/s eta 0:00:00
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.67.1
    Uninstalling tqdm-4.67.1:
      Successfully uninstalled tqdm-4.67.1
  Attempting uninstall: regex
    Found existing installation: regex 2024.11.6
    Uninstalling regex-2024.11.6:
      Successfully uninstalled regex-2024.11.6
  Attempting uninstall: joblib
    Found existing installation: joblib 1.4.2
    Uninstalling joblib-1.4.2:
      Successfully uninstalled

In [ ]:
import torch
import pandas as pd
import numpy as np
import nltk
import re
from bs4 import BeautifulSoup

# Ensure NLTK is correctly installed and restart the kernel
nltk.download('wordnet')

pd.set_option('display.max_colwidth', None)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
url = "https://web.archive.org/web/20201127142707if_/https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Office_Products_v1_00.tsv.gz"
df = pd.read_csv(url, sep='\t', compression='gzip', usecols=['review_body', 'star_rating'],on_bad_lines='skip')


KeyboardInterrupt: 

In [ ]:
#Ensure that star_rating contains only valid numeric values:

# Convert star_rating to numeric, coercing errors to NaN
df['star_rating'] = pd.to_numeric(df['star_rating'], errors='coerce')

# Drop rows with invalid star_rating values
df = df.dropna(subset=['star_rating'])

# Convert star_rating to integer (if required)
df['star_rating'] = df['star_rating'].astype(int)

In [ ]:
df

In [ ]:
balanced_df = df.groupby('star_rating').apply(lambda x: x.sample(n=50000, random_state=42)).reset_index(drop=True)
balanced_df

In [ ]:
def map_sentiment(rating):
    if rating > 3:
        return 1  # Positive
    elif rating < 3:
        return 2  # Negative
    else:
        return 3  # Neutral

balanced_df['sentiment'] = balanced_df['star_rating'].apply(map_sentiment)
orig_df = balanced_df.copy()
balanced_df

In [ ]:
#train_df, test_df = train_test_split(balanced_df, test_size=0.2, stratify=balanced_df['sentiment'], random_state=42)
print(orig_df)

In [ ]:
pip install gensim

In [ ]:
import pandas as pd
import gensim
from gensim.models import Word2Vec, KeyedVectors
from nltk.tokenize import word_tokenize
import numpy as np
import nltk

# Download NLTK tokenizer
nltk.download('punkt')
import gensim.downloader as api
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
import nltk

# Download tokenizer
nltk.download('punkt')


In [ ]:
# Load pretrained Word2Vec model (Google News 300)
wv = api.load("word2vec-google-news-300")

# Check the vector representation of a word
#print(wv["excellent"])  # A 300-dimensional vector

In [ ]:
#check semantic similarities
result = wv.most_similar(positive=["king", "woman"], negative=["man"], topn=1)
print("King - Man + Woman =", result[0][0])  # Expected output: 'queen'

In [ ]:
#similarities between words
similarity = wv.similarity("excellent", "outstanding")
print("Similarity between 'excellent' and 'outstanding':", similarity)

In [ ]:
import nltk
nltk.download('punkt_tab')
#tokenize your dataset
# Tokenize the review text
balanced_df["tokens"] = balanced_df["review_body"].astype(str).apply(word_tokenize)
# Show some tokenized reviews
print(balanced_df["tokens"].head())


In [ ]:
#converting to word embeddings
'''
def get_word2vec_embedding(tokens, model):
    vectors = [model[word] for word in tokens if word in model]
    return np.mean(vectors, axis=0) if vectors else np.zeros(300)  # 300D vector

# Applying to dataset using the pretrained Word2Vec model
balanced_df["embedding"] = balanced_df["tokens"].apply(lambda x: get_word2vec_embedding(x, wv))'''
import gensim.downloader as api
import numpy as np
import pandas as pd

# Load the pretrained Word2Vec model
wv = api.load("word2vec-google-news-300")

# Function to get embeddings for each word in a review
def get_word_embeddings(tokens, model):
    return [model[word] for word in tokens if word in model]  # List of word embeddings

# Load dataset (assume it's preprocessed and tokenized)
#df = pd.read_pickle("amazon_reviews.pkl")  # Replace with actual dataset file

# Apply function to dataset
balanced_df["word_embeddings"] = balanced_df["tokens"].apply(lambda x: get_word_embeddings(x, wv))

###(b) Training Word2Vec model on our own datset

In [ ]:
# Training my custom Word2Vec model
custom_w2v = Word2Vec(
    sentences=balanced_df["tokens"],  # List of tokenized reviews
    vector_size=300,         # Word embedding size
    window=11,               # Context window size
    min_count=10,            # Minimum word frequency threshold
    workers=4                # Use multiple CPU cores for efficiency
)

# Save the trained model
custom_w2v.save("custom_word2vec.model")

# Example: Check words most similar to "excellent"
print(custom_w2v.wv.most_similar("excellent"))

In [ ]:
result_custom = custom_w2v.wv.most_similar(positive=["king", "woman"], negative=["man"], topn=1)
print("Custom Word2Vec: King - Man + Woman =", result_custom[0][0])

In [ ]:
similarity_custom = custom_w2v.wv.similarity("excellent", "outstanding")
print("Custom Word2Vec similarity (excellent vs outstanding):", similarity_custom)

**Comparison based on King - Man + Woman on above 2 models :**

Google News Word2Vec : Returns "Queen"

Custom Word2Vec (Amazon Reviews) : Does not return "Queen" as the dataset may lacks historical context.


**Comparison based on Excellent vs Outstanding :**


Google News Word2Vec : High similarity (~0.9)


Custom Word2Vec (Amazon Reviews) :0.86168975 ( lower due to limited data.)


**Based on Context Understanding**

Google News Word2Vec : General knowledge	Biased towards Amazon review words.

Custom Word2Vec (Amazon Reviews): Biased towards Amazon review words.

**Conclusion:-**

Pretrained Word2Vec (Google News 300) performs better for general language tasks

It was trained on a large-scale dataset (Google News corpus).
It encodes rich semantic relationships between words.
It correctly captures King - Man + Woman = Queen.

Custom Word2Vec is more domain-specific (Amazon Reviews)

It is trained on customer reviews, so it understands sentiment-related words well.
But it lacks general world knowledge, making it weaker for analogy tasks.
Example: It may not return "Queen" for King - Man + Woman, but it might capture sentiment-related similarities better (e.g., "great" ∼ "awesome")

**Best Choice?**

If we need general word understanding, use then pretrained Word2Vec.

If we need domain-specific representations (Amazon reviews, legal, medical, etc.),  our own custom trained Word2V on reviews works better

In [ ]:
print(custom_w2v.wv.most_similar("excellent"))

### 3.

### Data cleaning and Data Preprocessing

In [ ]:
contractions_dict = {
    "aren't": "are not",
    "can't": "cannot",
    "couldn't": "could not",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hasn't": "has not",
    "haven't": "have not",
    "he's": "he is",
    "how's": "how is",
    "i'd": "I would",
    "i'll": "I will",
    "i'm": "I am",
    "i've": "I have",
    "isn't": "is not",
    "it's": "it is",
    "let's": "let us",
    "mightn't": "might not",
    "mustn't": "must not",
    "shan't": "shall not",
    "she's": "she is",
    "shouldn't": "should not",
    "that's": "that is",
    "there's": "there is",
    "they'd": "they would",
    "they'll": "they will",
    "they're": "they are",
    "they've": "they have",
    "we'd": "we would",
    "we're": "we are",
    "weren't": "were not",
    "what's": "what is",
    "where's": "where is",
    "who's": "who is",
    "won't": "will not",
    "wouldn't": "would not",
    "you'd": "you would",
    "you'll": "you will",
    "you're": "you are",
    "you've": "you have"
}

In [ ]:
df=orig_df.copy()
df

In [ ]:
#from locale import D_FMT
import pandas as pd
import re
import nltk
from nltk.corpus import words
import re
#from spellchecker import SpellChecker
import pandas as pd
import re

# Initialize spell checker
#spell = SpellChecker()

# Function to filter misspelled words
def remove_misspelled_words(text):
    if isinstance(text, str):
        # Split text into words
        words = text.split()
        # Check each word and retain it only if it's correctly spelled
        corrected_words = [word for word in words if word in spell]
        return " ".join(corrected_words)
    return ""

# Download the English words corpus (only required once)
nltk.download('words')

# Get the set of valid English words
english_words = set(words.words())

# Function to filter non-English words from a string
def filter_non_english_words(text):
    if isinstance(text, str):
        # Split text into words, filter non-English words, and rejoin
        filtered_words = [word for word in text.split() if word.lower() in english_words]
        return " ".join(filtered_words)
    return ""

def expand_contractions(text, contractions_map):
    if isinstance(text, str):
        pattern = re.compile(r'\b(' + '|'.join(re.escape(key) for key in contractions_map.keys()) + r')\b')
        return pattern.sub(lambda x: contractions_map[x.group()], text)
    return text

def preprocess_reviews(df):
    # Step 1: Convert all reviews to lowercase
    df['cleaned_review'] = df['review_body'].str.lower()

    # Step 2: Remove HTML tags and URLs
    df['cleaned_review'] = df['cleaned_review'].apply(lambda x: re.sub(r'<.*?>', '', x) if isinstance(x, str) else '')
    df['cleaned_review'] = df['cleaned_review'].apply(lambda x: re.sub(r'http\S+', '', x) if isinstance(x, str) else '')

    # Step 3: Expand contractions (e.g., can't -> can not)
    #df['cleaned_review'] = df['cleaned_review'].apply(lambda x: contractions.fix(x))
    df['cleaned_review'] = df['cleaned_review'].apply(lambda x: expand_contractions(x, contractions_dict))


    # Step 4: Separate concatenated words (e.g., slot...can't -> slot can't)
    df['cleaned_review'] = df['cleaned_review'].apply(lambda x: re.sub(r'(\w)(\'\w)', r'\1 \2', x))  # Insert space between word and contraction

    # Step 5: Replace multiple dots (two or more) with a space
    df['cleaned_review'] = df['cleaned_review'].apply(lambda x: re.sub(r'\.{2,}', ' ', x))

    # Step 6: Replace non-alphabetical characters (except for apostrophes and spaces) with a space
    #df['cleaned_review'] = df['cleaned_review'].apply(lambda x: re.sub(r'[^\w\s\'-]', ' ', x))  # Keep alphabet, spaces, apostrophes, and hyphens
    df['cleaned_review'] = df['cleaned_review'].apply(lambda x: re.sub(r'[^a-zA-Z\s\']', ' ', x))  # Keep only alphabet, spaces, and apostrophes
    # Step 7: Remove extra spaces (e.g., from replacing punctuation)
    df['cleaned_review'] = df['cleaned_review'].apply(lambda x: re.sub(r'\s+', ' ', x).strip())

    # Step 8:  Apply the function to remove misspelled words
    df['cleaned_review'] = df['cleaned_review'].apply(filter_non_english_words)
    #df['cleaned_review'] = df['cleaned_review'].apply(remove_misspelled_words)
    return df

def average_length(df, column='review_body'):
    # Replace non-string values with empty string
    df[column] = df[column].fillna('').astype(str)

    # Calculate average length
    lengths = df[column].apply(len)
    return lengths.mean()



# Calculate average lengths before preprocessing
avg_length_before = average_length(df, column='review_body')

# Preprocess the reviews
reduced_df = preprocess_reviews(df)

# Calculate average lengths after preprocessing
avg_length_after = average_length(df, column='cleaned_review')

# Output average lengths before and after
print(f"Average length before Data Cleaning: {avg_length_before}, Average length after Data Cleaning: {avg_length_after}")



#Average length before cleaning: 319.694505
#Average length after cleaning: 260.67447

In [ ]:
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

# Download necessary resources if not already downloaded
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

# Initialize the lemmatizer and stopwords
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

# Function to get the WordNet POS tag from the NLTK POS tag
def get_wordnet_pos(tag):
    if tag.startswith('VB'):  # Verbs
        return wordnet.VERB
    elif tag.startswith('JJ'):  # Adjectives
        return wordnet.ADJ
    elif tag.startswith('NN'):  # Nouns
        return wordnet.NOUN
    elif tag.startswith('RB'):  # Adverbs
        return wordnet.ADV
    else:
        return wordnet.NOUN  # Default to noun if no match

# Function to lemmatize text with POS tagging
def lemmatize_text(text):
    # Tokenize the sentence and POS tag each token
    tokens = word_tokenize(text)
    tagged_tokens = nltk.pos_tag(tokens)

    # Lemmatize using POS tags and filter out stopwords
    lemmatized_words = [
        lemmatizer.lemmatize(word, get_wordnet_pos(tag))
        for word, tag in tagged_tokens if word.lower() not in stop_words
    ]

    # Return the lemmatized sentence as a string
    return ' '.join(lemmatized_words)

# Apply lemmatization to your dataframe column
df['cleaned_review'] = df['cleaned_review'].apply(lemmatize_text)

In [ ]:
print(f"Average length before data preprocessing: {avg_length_after}, Average length after data preprocessing: {average_length(reduced_df, column='cleaned_review')}")


In [ ]:
df_3classes = df.copy()
# Labels: Sentiment classification (Class 1 = Positive, Class 2 = Negative)
df = df[df["sentiment"].isin([1, 2])]  # Ignore Class 3 (Neutral)

In [ ]:
df_3classes.shape

In [ ]:
filtered_df = df[df['sentiment'] == 5]
print(filtered_df)

In [ ]:
### Computing Word2Vec Embeddings from Google and Custom
from gensim.models import Word2Vec
import gensim.downloader as api

# Load pretrained Word2Vec (Google News 300)
wv_google = api.load("word2vec-google-news-300")

# Load custom-trained Word2Vec
custom_w2v = Word2Vec.load("custom_word2vec.model")

# Function to compute average word embedding for a review
def get_avg_embedding(tokens, model):
    vectors = [model[word] for word in tokens.split() if word in model]
    return np.mean(vectors, axis=0) if vectors else np.zeros(300)  # Return zero vector if no valid words

# Compute embeddings for both models
df["embedding_google"] = df["cleaned_review"].apply(lambda x: get_avg_embedding(x, wv_google))
df["embedding_custom"] = df["cleaned_review"].apply(lambda x: get_avg_embedding(x, custom_w2v.wv))

In [ ]:
s="uiuiu hjhj jhj"
for word in s.split():
  print(word)

In [ ]:
### Tf-idf feature extraction
newdf= reduced_df
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

# Initialize the TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(
    max_features=10000,  # Limit to top 5000 terms (optional)
    stop_words='english',  # Remove common stop words
    #ngram_range=(1,2)  # Use unigrams and bigrams
)
# Fit the vectorizer and transform the data
tfidf_features = tfidf_vectorizer.fit_transform(df['cleaned_review'])

In [ ]:
tfidf_features

In [ ]:
# Train - Test Split
from sklearn.model_selection import train_test_split
# Train-test split (80% training, 20% testing)
X_train_google, X_test_google, y_train, y_test = train_test_split(
    list(df["embedding_google"]), df["sentiment"], test_size=0.2, random_state=42, stratify=df["sentiment"]
)

X_train_custom, X_test_custom, _, _ = train_test_split(
    list(df["embedding_custom"]), df["sentiment"], test_size=0.2, random_state=42, stratify=df["sentiment"]
)

X_train_tfidf, X_test_tfidf, _, _ = train_test_split(
    tfidf_features, df["sentiment"], test_size=0.2, random_state=42, stratify=df["sentiment"]
)

In [ ]:
print(type(X_train_google))  # <class 'list'>
print(type(X_train_google[0]))  # <class 'numpy.ndarray'>
print(X_train_google[0].shape)  # (300,)

In [ ]:
print(type(X_train_custom))  # <class 'list'>
print(type(X_train_custom[0]))  # <class 'numpy.ndarray'>
print(X_train_custom[0].shape)  # (300,)

In [ ]:
#Perceptron training
from sklearn.linear_model import Perceptron
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Function to evaluate model performance
def evaluate_model(model, X_train, X_test, y_train, y_test, model_name):
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    # Training metrics
    train_acc = accuracy_score(y_train, y_train_pred)
    train_precision = precision_score(y_train, y_train_pred, average="binary")
    train_recall = recall_score(y_train, y_train_pred, average="binary")
    train_f1 = f1_score(y_train, y_train_pred, average="binary")

    # Test metrics
    test_acc = accuracy_score(y_test, y_test_pred)
    test_precision = precision_score(y_test, y_test_pred, average="binary")
    test_recall = recall_score(y_test, y_test_pred, average="binary")
    test_f1 = f1_score(y_test, y_test_pred, average="binary")

    # Print results
    print(f"\n📌 {model_name} Results:")
    print(f"✅ Training Accuracy: {train_acc:.4f} | Test Accuracy: {test_acc:.4f}")
    print(f"✅ Training Precision: {train_precision:.4f} | Test Precision: {test_precision:.4f}")
    print(f"✅ Training Recall: {train_recall:.4f} | Test Recall: {test_recall:.4f}")
    print(f"✅ Training F1 Score: {train_f1:.4f} | Test F1 Score: {test_f1:.4f}")

    #return {
    #    "train_acc": train_acc, "test_acc": test_acc,
    #    "train_precision": train_precision, "test_precision": test_precision,
    #    "train_recall": train_recall, "test_recall": test_recall,
    #    "train_f1": train_f1, "test_f1": test_f1
    #}

### **1️⃣ Perceptron Models**
print("\n🔹 Training Perceptron Models...")
perc_google = Perceptron()
perc_google.fit(X_train_google, y_train)
evaluate_model(perc_google, X_train_google, X_test_google, y_train, y_test, "Perceptron (Google News W2V)")

perc_custom = Perceptron()
perc_custom.fit(X_train_custom, y_train)
evaluate_model(perc_custom, X_train_custom, X_test_custom, y_train, y_test, "Perceptron (Custom W2V)")

perc_tfidf = Perceptron()
perc_tfidf.fit(X_train_tfidf, y_train)
evaluate_model(perc_tfidf, X_train_tfidf, X_test_tfidf, y_train, y_test, "Perceptron (TF-IDF)")


### **2️⃣ SVM Models**
print("\n🔹 Training SVM Models...")
svm_google = LinearSVC(random_state=42, max_iter=10000)
svm_google.fit(X_train_google, y_train)
evaluate_model(svm_google, X_train_google, X_test_google, y_train, y_test, "SVM (Google News W2V)")

svm_custom = LinearSVC(random_state=42, max_iter=10000)
svm_custom.fit(X_train_custom, y_train)
evaluate_model(svm_custom, X_train_custom, X_test_custom, y_train, y_test, "SVM (Custom W2V)")

svm_tfidf =LinearSVC(random_state=42, max_iter=10000)
svm_tfidf.fit(X_train_tfidf, y_train)
evaluate_model(svm_tfidf, X_train_tfidf, X_test_tfidf, y_train, y_test, "SVM (TF-IDF)")


### TF-IDF remains the best performing feature type for both Perceptron and SVM, achieving: Best test accuracy (84.1%) with SVM.

### SVM outperforms Perceptron consistently.

### Custom Word2Vec benefits from domain-specific training but still lags behind TF-IDF.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import gensim.downloader as api
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


In [ ]:
df = df_3classes.copy()
df = df[df["sentiment"].isin([1, 2])]
from sklearn.model_selection import train_test_split

### Computing Word2Vec Embeddings from Google and Custom
from gensim.models import Word2Vec
import gensim.downloader as api

# Load pretrained Word2Vec (Google News 300)
wv_google = api.load("word2vec-google-news-300")

# Load custom-trained Word2Vec
custom_w2v = Word2Vec.load("custom_word2vec.model")

# Function to compute average word embedding for a review
def get_avg_embedding(tokens, model):
    vectors = [model[word] for word in tokens.split() if word in model]
    return np.mean(vectors, axis=0) if vectors else np.zeros(300)  # Return zero vector if no valid words

# Compute embeddings for both models
df["embedding_google"] = df["cleaned_review"].apply(lambda x: get_avg_embedding(x, wv_google))
df["embedding_custom"] = df["cleaned_review"].apply(lambda x: get_avg_embedding(x, custom_w2v.wv))



In [ ]:
# Train-test split (80% training, 20% testing)
X_train_google, X_test_google, y_train, y_test = train_test_split(
    list(df["embedding_google"]), df["sentiment"], test_size=0.2, random_state=42, stratify=df["sentiment"]
)

X_train_custom, X_test_custom, _, _ = train_test_split(
    list(df["embedding_custom"]), df["sentiment"], test_size=0.2, random_state=42, stratify=df["sentiment"]
)

#For ternary :
df_3classes["embedding_google"] = df_3classes["cleaned_review"].apply(lambda x: get_avg_embedding(x, wv_google))
df_3classes["embedding_custom"] = df_3classes["cleaned_review"].apply(lambda x: get_avg_embedding(x, custom_w2v.wv))


# Train-test split (80% training, 20% testing)
X_train_google_ternary, X_test_google_ternary, y_train_ter, y_test_ter = train_test_split(
    list(df_3classes["embedding_google"]), df_3classes["sentiment"], test_size=0.2, random_state=42, stratify=df_3classes["sentiment"]
)

X_train_custom_ternary, X_test_custom_ternary, _, _ = train_test_split(
    list(df_3classes["embedding_custom"]), df_3classes["sentiment"], test_size=0.2, random_state=42, stratify=df_3classes["sentiment"]
)


In [ ]:
y_train_binary = np.array(y_train.replace({1: 1, 2: 0}))
y_test_binary = np.array(y_test.replace({1: 1, 2: 0}))

In [ ]:
y_train_ternary = np.array(y_train_ter .replace({1: 0, 2: 1,3:2}))
y_test_ternary = np.array(y_test_ter.replace({1: 0, 2: 1,3:2}))

In [ ]:
#creating a Feedforward MLP with two hidden layers (50 & 10 nodes).
class MLP(nn.Module):
    def __init__(self, input_size, num_classes):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)  # First hidden layer (50 neurons)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(50, 10)  # Second hidden layer (10 neurons)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(10, num_classes)  # Output layer

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        return x


In [ ]:
def train_model(X_train, y_train, X_test, y_test, num_classes, epochs=250, lr=0.001):
    # Convert numpy arrays to PyTorch tensors
    X_train, y_train = torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.long)
    X_test, y_test = torch.tensor(X_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.long)

    # Define model
    model = MLP(input_size=300, num_classes=num_classes)
    criterion = nn.CrossEntropyLoss()  # Cross-entropy loss
    optimizer = optim.Adam(model.parameters(), lr=lr)  # Adam optimizer

    # Training loop
    for epoch in range(epochs):
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = criterion(outputs, y_train)
        loss.backward()
        optimizer.step()

        if (epoch + 1) % 5 == 0:
            print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

    # Evaluate on test set
    with torch.no_grad():
        test_outputs = model(X_test)
        _, predicted = torch.max(test_outputs, 1)
        accuracy = accuracy_score(y_test.numpy(), predicted.numpy())

    return accuracy


In [ ]:
# Training Binary Classification Models:
print("Binary Classification (Positive vs Negative)")

# Train on Google Word2Vec
acc_binary_google = train_model(X_train_google, y_train_binary, X_test_google, y_test_binary, num_classes=2)
print()
print(f"✅ Google W2V Accuracy (Binary): {acc_binary_google:.4f}")
print()
# Train on Custom Word2Vec
acc_binary_custom = train_model(X_train_custom, y_train_binary, X_test_custom, y_test_binary, num_classes=2)
print(f"✅ Custom W2V Accuracy (Binary): {acc_binary_custom:.4f}")




In [ ]:
print("Ternary Classification (Positive, Negative, Neutral)")
#X_train_google_ternary, X_test_google_ternary
#X_train_custom_ternary, X_test_custom_ternary
# Train on Google Word2Vec
acc_ternary_google = train_model(X_train_google_ternary, y_train_ternary, X_test_google_ternary, y_test_ternary, num_classes=3)
print(f"✅ Google W2V Accuracy (Ternary): {acc_ternary_google:.4f}")

# Train on Custom Word2Vec
acc_ternary_custom = train_model(X_train_custom_ternary, y_train_ternary, X_test_custom_ternary, y_test_ternary, num_classes=3)
print(f"✅ Custom W2V Accuracy (Ternary): {acc_ternary_custom:.4f}")


###(b)

In [ ]:
def concat_w2v_vectors(tokens, model, num_vectors=10):
    # Use model[word] directly for KeyedVectors (Google W2V) and model.wv[word] for Word2Vec models
    vectors = [model[word] if isinstance(model, gensim.models.KeyedVectors) else model.wv[word] for word in tokens if word in model]

    # If fewer than num_vectors words exist, pad with zero vectors
    while len(vectors) < num_vectors:
        vectors.append(np.zeros(model.vector_size))

    # Take only the first num_vectors
    vectors = vectors[:num_vectors]

    # Flatten (concatenate) the vectors
    return np.concatenate(vectors, axis=0)

# Convert dataset to concatenated vectors
X_google_concat = np.array([concat_w2v_vectors(tokens, wv_google) for tokens in df["cleaned_review"]])
X_custom_concat = np.array([concat_w2v_vectors(tokens, custom_w2v.wv) for tokens in df["cleaned_review"]])



In [ ]:
print("Training Data Shape:", X_google_concat.shape)

In [ ]:
#print("Ternary training Data Shape:", X_google_concat_tern.shape)

In [ ]:
X_google_concat = X_google_concat.astype(np.float32)
X_custom_concat = X_custom_concat.astype(np.float32)



In [ ]:
from sklearn.model_selection import train_test_split

# Binary classification split (one at a time)
X_train_google_concat_binary, X_test_google_concat_binary, y_train_google_concat_binary, y_test_google_concat_binary = train_test_split(
    X_google_concat, df['sentiment'], test_size=0.2, random_state=42, stratify=df['sentiment'])

#del X_google_concat  # Free memory

X_train_custom_concat_binary, X_test_custom_concat_binary, y_train_custom_concat_binary, y_test_custom_concat_binary = train_test_split(
    X_custom_concat, df['sentiment'], test_size=0.2, random_state=42, stratify=df['sentiment'])

#del X_custom_concat  # Free memory





In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#model.to(device)

In [ ]:
'''def train_model_concat(X_train, y_train, X_test, y_test, num_classes, epochs=160, lr=0.005):
    # ✅ Convert Pandas Series to NumPy before torch.tensor()

    if(num_classes==2):
      y_train_binary = np.array(y_train.replace({1: 1, 2: 0}))
      y_test_binary = np.array(y_test.replace({1: 1, 2: 0}))
      X_train, y_train = torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train_binary, dtype=torch.long)
      X_test, y_test = torch.tensor(X_test, dtype=torch.float32), torch.tensor(y_test_binary , dtype=torch.long)
    elif(num_classes==3):
      y_train_ternary = np.array(y_train_ter .replace({1: 0, 2: 1,3:2}))
      y_test_ternary = np.array(y_test_ter.replace({1: 0, 2: 1,3:2}))
      X_train, y_train = torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train_ternary, dtype=torch.long)
      X_test, y_test = torch.tensor(X_test, dtype=torch.float32), torch.tensor(y_test_ternary , dtype=torch.long)
    model.to(device)
    # Define the model
    model = MLP(input_size=3000, num_classes=num_classes)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    # Training loop
    for epoch in range(epochs):
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = criterion(outputs, y_train)
        loss.backward()
        optimizer.step()

        if (epoch + 1) % 5 == 0:
            print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

    # Evaluation
    # Evaluation on the training set
    with torch.no_grad():
        train_outputs = model(X_train)
        _, train_predicted = torch.max(train_outputs, 1)
        train_accuracy = accuracy_score(y_train.numpy(), train_predicted.numpy())

        # Evaluation on the testing set
        test_outputs = model(X_test)
        _, test_predicted = torch.max(test_outputs, 1)
        test_accuracy = accuracy_score(y_test.numpy(), test_predicted.numpy())

    return train_accuracy, test_accuracy
'''
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.metrics import accuracy_score

def train_model_concat(X_train, y_train, X_test, y_test, num_classes, epochs=160, lr=0.005):
    # ✅ Move tensors to GPU if available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # ✅ Convert Pandas Series to NumPy before torch.tensor()
    if num_classes == 2:
        y_train_binary = np.array(y_train.replace({1: 1, 2: 0}))
        y_test_binary = np.array(y_test.replace({1: 1, 2: 0}))
        X_train, y_train = torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train_binary, dtype=torch.long)
        X_test, y_test = torch.tensor(X_test, dtype=torch.float32), torch.tensor(y_test_binary, dtype=torch.long)

    elif num_classes == 3:
        y_train_ternary = np.array(y_train.replace({1: 0, 2: 1, 3: 2}))  # ✅ Fix variable name
        y_test_ternary = np.array(y_test.replace({1: 0, 2: 1, 3: 2}))  # ✅ Fix variable name
        X_train, y_train = torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train_ternary, dtype=torch.long)
        X_test, y_test = torch.tensor(X_test, dtype=torch.float32), torch.tensor(y_test_ternary, dtype=torch.long)

    # ✅ Define the model first
    model = MLP(input_size=3000, num_classes=num_classes)

    # ✅ Move the model to GPU
    model.to(device)

    # ✅ Move input tensors to the same device as the model
    X_train, y_train = X_train.to(device), y_train.to(device)
    X_test, y_test = X_test.to(device), y_test.to(device)

    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    # Training loop
    for epoch in range(epochs):
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = criterion(outputs, y_train)
        loss.backward()
        optimizer.step()

        if (epoch + 1) % 5 == 0:
            print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

    # Evaluation
    with torch.no_grad():
        train_outputs = model(X_train)
        _, train_predicted = torch.max(train_outputs, 1)
        train_accuracy = accuracy_score(y_train.cpu().numpy(), train_predicted.cpu().numpy())  # ✅ Move to CPU before converting to NumPy

        test_outputs = model(X_test)
        _, test_predicted = torch.max(test_outputs, 1)
        test_accuracy = accuracy_score(y_test.cpu().numpy(), test_predicted.cpu().numpy())  # ✅ Move to CPU before converting to NumPy

    return train_accuracy, test_accuracy



In [ ]:
# Binary Classification (Concatenated Word2Vec)
print("Binary Classification (Concatenated Word2Vec)")

# Google Word2Vec
train_acc_google_concat, test_acc_google_concat = train_model_concat(
    X_train_google_concat_binary, y_train_google_concat_binary,
    X_test_google_concat_binary, y_test_google_concat_binary,
    num_classes=2
)
print(f"✅  Google W2V - Train Accuracy (Binary - Concat): {train_acc_google_concat:.4f}")
print(f"✅  Google W2V - Test Accuracy (Binary - Concat): {test_acc_google_concat:.4f}")

# Custom Word2Vec
train_acc_custom_concat, test_acc_custom_concat = train_model_concat(
    X_train_custom_concat_binary, y_train_custom_concat_binary,
    X_test_custom_concat_binary, y_test_custom_concat_binary,
    num_classes=2
)
print(f"✅ Custom W2V - Train Accuracy (Binary - Concat): {train_acc_custom_concat:.4f}")
print(f"✅ Custom W2V - Test Accuracy (Binary - Concat): {test_acc_custom_concat:.4f}")

In [ ]:
#del X_google_concat
#del X_custom_concat
#Similarly for Ternary , get the 3000 input vector size:
# Convert dataset to concatenated vectors
#X_google_concat_tern = X_google_concat_tern.astype(np.float32)
#X_custom_concat_tern = X_custom_concat_tern.astype(np.float32)
X_google_concat_tern = np.array([concat_w2v_vectors(tokens, wv_google) for tokens in df_3classes["cleaned_review"]])
X_custom_concat_tern = np.array([concat_w2v_vectors(tokens, custom_w2v.wv) for tokens in df_3classes["cleaned_review"]])


In [ ]:
# Ternary classification split

X_train_google_concat_ternary, X_test_google_concat_ternary, y_train_google_concat_ternary, y_test_google_concat_ternary = train_test_split(
    X_google_concat_tern, df_3classes['sentiment'], test_size=0.2, random_state=42, stratify=df_3classes['sentiment'])


X_train_custom_concat_ternary, X_test_custom_concat_ternary, y_train_custom_concat_ternary, y_test_custom_concat_ternary = train_test_split(
    X_custom_concat_tern, df_3classes['sentiment'], test_size=0.2, random_state=42, stratify=df_3classes['sentiment'])



In [ ]:
print("CUDA Available:", torch.cuda.is_available())
print("GPU Name:", torch.cuda.get_device_name(0))

In [ ]:
import gc
import torch

gc.collect()
torch.cuda.empty_cache()
# ternary Classification (Concatenated Word2Vec)
print("Ternary Classification (Concatenated Word2Vec)")

# Google Word2Vec
train_acc_google_concat_tern, test_acc_google_concat_tern = train_model_concat(
    X_train_google_concat_ternary, y_train_google_concat_ternary,
    X_test_google_concat_ternary, y_test_google_concat_ternary,
    num_classes=3
)
print(f"✅  Google W2V - Train Accuracy (Ternary - Concat): {train_acc_google_concat_tern:.4f}")
print(f"✅  Google W2V - Test Accuracy (Ternary - Concat): {test_acc_google_concat_tern:.4f}")

# Custom Word2Vec
train_acc_custom_concat_tern, test_acc_custom_concat_tern = train_model_concat(
    X_train_custom_concat_ternary, y_train_custom_concat_ternary,
    X_test_custom_concat_ternary, y_test_custom_concat_ternary,
    num_classes=3
)
print(f"✅ Custom W2V - Train Accuracy (Ternary - Concat): {train_acc_custom_concat_tern:.4f}")
print(f"✅ Custom W2V - Test Accuracy (Ternary - Concat): {test_acc_custom_concat_tern:.4f}")

In [ ]:
'''
#Splitting binary into train and test : 80-20
from sklearn.model_selection import train_test_split
# Binary
X_train_google_concat_binary, X_test_google_concat_binary, y_train_google_concat_binary, y_test_google_concat_binary = train_test_split(
    X_google_concat, df['sentiment'], test_size=0.2, random_state=42, stratify=df['sentiment'])

X_train_custom_concat_binary, X_test_custom_concat_binary, y_train_custom_concat_binary, y_test_custom_concat_binary = train_test_split(
    X_custom_concat, df['sentiment'], test_size=0.2, random_state=42, stratify=df['sentiment'])

#Ternary

X_train_google_concat_ternary, X_test_google_concat_ternary, y_train_google_concat_ternary, y_test_google_concat_ternary = train_test_split(
    X_google_concat_tern, df_3classes['sentiment'], test_size=0.2, random_state=42, stratify=df_3classes['sentiment'])

X_train_custom_concat_ternary, X_test_custom_concat_ternary, y_train_custom_concat_ternary, y_test_custom_concat_ternary = train_test_split(
    X_custom_concat_tern, df_3classes['sentiment'], test_size=0.2, random_state=42, stratify=df_3classes['sentiment'])
'''

In [ ]:
'''X_train_google, X_test_google, y_train, y_test = train_test_split(
    list(df["embedding_google"]), df["sentiment"], test_size=0.2, random_state=42, stratify=df["sentiment"]
)

X_train_custom, X_test_custom, _, _ = train_test_split(
    list(df["embedding_custom"]), df["sentiment"], test_size=0.2, random_state=42, stratify=df["sentiment"]
)

#For ternary :
df_3classes["embedding_google"] = df_3classes["cleaned_review"].apply(lambda x: get_avg_embedding(x, wv_google))
df_3classes["embedding_custom"] = df_3classes["cleaned_review"].apply(lambda x: get_avg_embedding(x, custom_w2v.wv))


# Train-test split (80% training, 20% testing)
X_train_google_ternary, X_test_google_ternary, y_train_ter, y_test_ter = train_test_split(
    list(df_3classes["embedding_google"]), df_3classes["sentiment"], test_size=0.2, random_state=42, stratify=df_3classes["sentiment"]
)

X_train_custom_ternary, X_test_custom_ternary, _, _ = train_test_split(
    list(df_3classes["embedding_custom"]), df_3classes["sentiment"], test_size=0.2, random_state=42, stratify=df_3classes["sentiment"]
)'''

### CNN

In [ ]:
df.shape

In [ ]:
'''import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import gensim.downloader as api
from gensim.models import Word2Vec
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import pandas as pd
# Convert embeddings into numpy arrays
X_google = np.stack(df["embedding_google"].values)
X_custom = np.stack(df["embedding_custom"].values)
y = df["sentiment"].values  # Assuming binary sentiment labels (0,1) or ternary (0,1,2)


# Split Data (Binary Classification)
X_train_google, X_test_google, y_train_google, y_test_google = train_test_split(
    X_google, y, test_size=0.2, random_state=42, stratify=y
)

X_train_custom, X_test_custom, y_train_custom, y_test_custom = train_test_split(
    X_custom, y, test_size=0.2, random_state=42, stratify=y
)

# Define CNN Model
class SentimentCNN(nn.Module):
    def __init__(self, input_size=300, num_classes=2):
        super(SentimentCNN, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=50, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(in_channels=50, out_channels=10, kernel_size=3, padding=1)
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)
        self.fc = nn.Linear(10 * 150, num_classes)  # Adjusted for input shape

    def forward(self, x):
        x = x.unsqueeze(1)  # Add channel dimension
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = self.fc(x)
        return x

# Function to Train and Evaluate CNN
def train_cnn_model(X_train, y_train, X_test, y_test, num_classes=2, epochs=10, batch_size=64, lr=0.001):
    # Convert to tensors
    X_train, y_train = torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.long)
    X_test, y_test = torch.tensor(X_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.long)

    # Create DataLoaders
    train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(TensorDataset(X_test, y_test), batch_size=batch_size, shuffle=False)


    # Initialize Model
    model = SentimentCNN(input_size=300, num_classes=num_classes)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    # Training Loop
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        print(f"Epoch [{epoch+1}/{epochs}], Loss: {total_loss:.4f}")

    # Evaluate Model
    model.eval()
    with torch.no_grad():
        # Train Accuracy
        train_preds = torch.cat([torch.argmax(model(X_batch), dim=1) for X_batch, _ in train_loader])
        train_acc = accuracy_score(y_train.numpy(), train_preds.numpy())

        # Test Accuracy
        test_preds = torch.cat([torch.argmax(model(X_batch), dim=1) for X_batch, _ in test_loader])
        test_acc = accuracy_score(y_test.numpy(), test_preds.numpy())

    return train_acc, test_acc

# Train CNN on Google Word2Vec Features
train_acc_google, test_acc_google = train_cnn_model(X_train_google, y_train_google, X_test_google, y_test_google, num_classes=2)
print(f"Google W2V - Train Accuracy (Binary - CNN): {train_acc_google:.4f}")
print(f"Google W2V - Test Accuracy (Binary - CNN): {test_acc_google:.4f}")

# Train CNN on Custom Word2Vec Features
train_acc_custom, test_acc_custom = train_cnn_model(X_train_custom, y_train_custom, X_test_custom, y_test_custom, num_classes=2)
print(f"Custom W2V - Train Accuracy (Binary - CNN): {train_acc_custom:.4f}")
print(f"Custom W2V - Test Accuracy (Binary - CNN): {test_acc_custom:.4f}")'''

In [ ]:
#main code
'''import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import ast

# ✅ Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
if device.type == "cuda":
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")

# ✅ Set parameters
max_review_length = 50  # Fixed review length (truncate/pad)
embedding_dim = 300     # Word embedding dimension
batch_size = 64
num_epochs = 10
learning_rate = 0.001

# ✅ Load dataset (ensure correct format)
# df = pd.read_csv("your_dataset.csv")  # Load dataset (modify path as needed)

# ✅ Function to safely convert embeddings
def safe_convert(embedding):
    try:
        return np.array(ast.literal_eval(embedding), dtype=np.float32)  # Safer parsing
    except (ValueError, SyntaxError):
        return np.zeros(300, dtype=np.float32)  # Handle corrupted data

# ✅ Convert 'embedding_google' column to NumPy array
X = np.array(df["embedding_google"].apply(safe_convert).tolist(), dtype=np.float32)
y = df["sentiment"].values  # Assuming 'sentiment' is the label column

# Initialize X_reshaped to have the right shape: (num_samples, 50, 300)
X_reshaped = np.zeros((len(X), max_review_length, embedding_dim), dtype=np.float32)

# Reshape each review to have a fixed length of 50 words, ensuring each embedding has shape (300,)
for i, review in enumerate(X):
    review_length = len(review)

    # Truncate longer reviews
    if review_length > max_review_length:
        X_reshaped[i] = review[:max_review_length]
    else:
        # Pad shorter reviews with zero vectors (each padding element is a vector of shape (300,))
        X_reshaped[i, :review_length] = review

# Verify shapes
print("X shape after reshaping:", X_reshaped.shape)  # Expected: (200000, 50, 300)
print("y shape:", y.shape)  # Expected: (200000,)


# Verify shapes
print("X shape after reshaping:", X_reshaped.shape)  # Should be (200000, 50, 300)
print("y shape:", y.shape)  # Should be (200000,)


# Verify shapes
print("X shape after reshaping:", X_reshaped.shape)  # Should be (200000, 50, 300)
print("y shape:", y.shape)  # Should be (200000,)

# ✅ Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_reshaped, y, test_size=0.2, random_state=42, stratify=y)

# ✅ Convert to PyTorch tensors and move to GPU
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train - 1, dtype=torch.long).to(device)  # Adjust labels to (0,1,2)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test - 1, dtype=torch.long).to(device)

# ✅ Add the missing channel dimension for CNN input
X_train_tensor = X_train_tensor.unsqueeze(1)  # Shape: (num_samples, 1, 50, 300)
X_test_tensor = X_test_tensor.unsqueeze(1)

# ✅ Print final tensor shapes
print("Final X_train_tensor shape:", X_train_tensor.shape)  # Expected: (num_samples, 1, 50, 300)
print("Final X_test_tensor shape:", X_test_tensor.shape)

# ✅ Create DataLoaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# ✅ Define CNN Model
class SentimentCNN(nn.Module):
    def __init__(self, num_classes=2):
        super(SentimentCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=50, kernel_size=(3, 3), padding=1)  # Output: (50, 50, 300)
        self.pool1 = nn.MaxPool2d(kernel_size=(2, 2), stride=2)  # Output: (50, 25, 150)
        self.conv2 = nn.Conv2d(in_channels=50, out_channels=10, kernel_size=(3, 3), padding=1)  # Output: (10, 25, 150)
        self.pool2 = nn.MaxPool2d(kernel_size=(2, 2), stride=2)  # Output: (10, 12, 75)

        # Compute the flattened feature size
        self.fc = nn.Linear(10 * 12 * 75, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = self.fc(x)
        return x

# ✅ Train and Evaluate Model
def train_cnn_model(num_classes=2):
    model = SentimentCNN(num_classes=num_classes).to(device)  # Move model to GPU
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Training Loop
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)  # Move batch to GPU
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss:.4f}")

    # Evaluate Model
    model.eval()
    with torch.no_grad():
        test_preds = torch.cat([torch.argmax(model(X_batch.to(device)), dim=1) for X_batch, _ in test_loader])
        test_acc = accuracy_score(y_test_tensor.cpu().numpy(), test_preds.cpu().numpy())

    print(f"Test Accuracy: {test_acc:.4f}")
    return model

# ✅ Train model for binary classification
print("Training CNN for Binary Classification...")
binary_model = train_cnn_model(num_classes=2)

# ✅ Train model for ternary classification
print("Training CNN for Ternary Classification...")
ternary_model = train_cnn_model(num_classes=3)'''

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import ast

# ✅ Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
if device.type == "cuda":
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")

# ✅ Set parameters
max_review_length = 50  # Fixed review length (truncate/pad)
embedding_dim = 300     # Word embedding dimension
batch_size = 64
num_epochs = 10
learning_rate = 0.001

# ✅ Load dataset (ensure correct format)
# df = pd.read_csv("your_dataset.csv")  # Load dataset (modify path as needed)

# ✅ Function to safely convert embeddings
def safe_convert(embedding):
    try:
        return np.array(ast.literal_eval(embedding), dtype=np.float32)  # Safer parsing
    except (ValueError, SyntaxError):
        return np.zeros((300,), dtype=np.float32)  # Handle corrupted data (return a 1D array of shape (300,))

# ✅ Convert 'embedding_google' column to NumPy array
X = np.array(df["embedding_google"].apply(safe_convert).tolist(), dtype=np.float32)
y = df["sentiment"].values  # Assuming 'sentiment' is the label column

# Initialize X_reshaped to have the right shape: (num_samples, 50, 300)
X_reshaped = np.zeros((len(X), max_review_length, embedding_dim), dtype=np.float32)

# Reshape each review to have a fixed length of 50 words, ensuring each embedding has shape (300,)
for i, review in enumerate(X):
    review_length = len(review)

    # Truncate longer reviews
    if review_length > max_review_length:
        X_reshaped[i] = review[:max_review_length]
    else:
        # Pad shorter reviews with zero vectors (each padding element is a vector of shape (300,))
        X_reshaped[i, :review_length] = review

# Verify shapes
print("X shape after reshaping:", X_reshaped.shape)  # Expected: (200000, 50, 300)
print("y shape:", y.shape)  # Expected: (200000,)

# ✅ Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_reshaped, y, test_size=0.2, random_state=42, stratify=y)

# ✅ Convert to PyTorch tensors and move to GPU
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train - 1, dtype=torch.long).to(device)  # Adjust labels to (0,1,2)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test - 1, dtype=torch.long).to(device)

# ✅ Add the missing channel dimension for CNN input
X_train_tensor = X_train_tensor.unsqueeze(1)  # Shape: (num_samples, 1, 50, 300)
X_test_tensor = X_test_tensor.unsqueeze(1)

# ✅ Print final tensor shapes
print("Final X_train_tensor shape:", X_train_tensor.shape)  # Expected: (num_samples, 1, 50, 300)
print("Final X_test_tensor shape:", X_test_tensor.shape)

# ✅ Create DataLoaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# ✅ Define CNN Model
class SentimentCNN(nn.Module):
    def __init__(self, num_classes=2):
        super(SentimentCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=50, kernel_size=(3, 3), padding=1)  # Output: (50, 50, 300)
        self.pool1 = nn.MaxPool2d(kernel_size=(2, 2), stride=2)  # Output: (50, 25, 150)
        self.conv2 = nn.Conv2d(in_channels=50, out_channels=10, kernel_size=(3, 3), padding=1)  # Output: (10, 25, 150)
        self.pool2 = nn.MaxPool2d(kernel_size=(2, 2), stride=2)  # Output: (10, 12, 75)

        # Compute the flattened feature size
        self.fc = nn.Linear(10 * 12 * 75, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = self.fc(x)
        return x

# ✅ Train and Evaluate Model
def train_cnn_model(num_classes=2):
    model = SentimentCNN(num_classes=num_classes).to(device)  # Move model to GPU
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Training Loop
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)  # Move batch to GPU
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss:.4f}")

    # Evaluate Model
    model.eval()
    with torch.no_grad():
        test_preds = torch.cat([torch.argmax(model(X_batch.to(device)), dim=1) for X_batch, _ in test_loader])
        test_acc = accuracy_score(y_test_tensor.cpu().numpy(), test_preds.cpu().numpy())

    print(f"Test Accuracy: {test_acc:.4f}")
    return model

# ✅ Train model for binary classification
print("Training CNN for Binary Classification...")
binary_model = train_cnn_model(num_classes=2)

# ✅ Train model for ternary classification
print("Training CNN for Ternary Classification...")
ternary_model = train_cnn_model(num_classes=3)

In [ ]:
# Trial and error for CNN:

In [1]:
import gensim.downloader as api
import numpy as np
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [49]:
# Load the Google News Word2Vec model (300D embeddings)

word2vec_model = api.load("word2vec-google-news-300")

In [26]:
word2vec_model

In [29]:
url = "https://web.archive.org/web/20201127142707if_/https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Office_Products_v1_00.tsv.gz"
df = pd.read_csv(url, sep='\t', compression='gzip', usecols=['review_body', 'star_rating'],on_bad_lines='skip')


<ipython-input-29-dd748172f966>:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(url, sep='\t', compression='gzip', usecols=['review_body', 'star_rating'],on_bad_lines='skip')


In [30]:
df1=df.copy()

In [52]:
df=df1.copy()

In [53]:
#Ensure that star_rating contains only valid numeric values:

# Convert star_rating to numeric, coercing errors to NaN
df['star_rating'] = pd.to_numeric(df['star_rating'], errors='coerce')

# Drop rows with invalid star_rating values
df = df.dropna(subset=['star_rating'])

# Convert star_rating to integer (if required)
df['star_rating'] = df['star_rating'].astype(int)
def map_sentiment(rating):
    if rating > 3:
        return 1  # Positive
    elif rating < 3:
        return 2  # Negative
    else:
        return 3  # Neutral

df['sentiment'] = df['star_rating'].apply(map_sentiment)
df

<ipython-input-53-8cffc3a0cd0f>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['star_rating'] = df['star_rating'].astype(int)


,star_rating,review_body,sentiment
0,5,Great product.,1
1,5,What's to say about this commodity item except...,1
2,5,"Haven't used yet, but I am sure I will like it.",1
3,1,Although this was labeled as &#34;new&#34; the...,2
4,4,Gorgeous colors and easy to use,1
...,...,...,...
2640347,4,I can't live anymore whithout my Palm III. But...,1
2640348,4,Although the Palm Pilot is thin and compact it...,1
2640349,4,This book had a lot of great content without b...,1
2640350,5,I am teaching a course in Excel and am using t...,1


In [54]:
df=df.groupby('star_rating').apply(lambda x: x.sample(n=50000, random_state=42)).reset_index(drop=True)
df

<ipython-input-54-238ddc82b0f8>:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df=df.groupby('star_rating').apply(lambda x: x.sample(n=50000, random_state=42)).reset_index(drop=True)


,star_rating,review_body,sentiment
0,1,This item was absolutely terrible. The ink bar...,2
1,1,out from the box there is a blank area when it...,2
2,1,I read ALL the reviews Bad & Good and made my ...,2
3,1,This came very sharp and it would not come apa...,2
4,1,I'd give this day timer zero stars if that was...,2
...,...,...,...
249995,5,LO QUE BUSCABA PARA MI OFICINA EN CASA MUY BUE...,1
249996,5,I have had Panasonic 3 phones for a long time....,1
249997,5,These are great pens to draw with and I like t...,1
249998,5,"I do martial arts, Jiu-Jitsu and I like to tak...",1


In [55]:
df_3 = df.copy()
#for binary
df
import nltk
nltk.download('punkt_tab')
def preprocess_text(text):
    text = str(text).lower()  # Convert to lowercase
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove special characters
    tokens = word_tokenize(text)  # Tokenize words
    return [word for word in tokens if word in word2vec_model]  # Keep only words in Word2Vec vocabulary

# Apply preprocessing to all reviews
df["cleaned_review"] = df["review_body"].apply(preprocess_text)

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [56]:
def get_word_embeddings(words, model):
    return [model[word] for word in words if word in model]  # Get embeddings for each word

# Apply function to dataset
df["word_embeddings"] = df["cleaned_review"].apply(lambda x: get_word_embeddings(x, word2vec_model))


In [ ]:
df = df[df['sentiment'] != 3]
df = df.sample(frac=1, random_state=42).iloc[:150000]

In [57]:
df

,star_rating,review_body,sentiment,cleaned_review,word_embeddings
0,1,This item was absolutely terrible. The ink bar...,2,"[this, item, was, absolutely, terrible, the, i...","[[0.109375, 0.140625, -0.03173828, 0.16601562,..."
1,1,out from the box there is a blank area when it...,2,"[out, from, the, box, there, is, blank, area, ...","[[0.110839844, -0.15820312, 0.020751953, 0.166..."
2,1,I read ALL the reviews Bad & Good and made my ...,2,"[i, read, all, the, reviews, bad, good, made, ...","[[-0.22558594, -0.01953125, 0.09082031, 0.2373..."
3,1,This came very sharp and it would not come apa...,2,"[this, came, very, sharp, it, would, not, come...","[[0.109375, 0.140625, -0.03173828, 0.16601562,..."
4,1,I'd give this day timer zero stars if that was...,2,"[id, give, this, day, timer, zero, stars, if, ...","[[0.29492188, -0.12109375, 0.0006828308, 0.220..."
...,...,...,...,...,...
249995,5,LO QUE BUSCABA PARA MI OFICINA EN CASA MUY BUE...,1,"[lo, que, para, mi, oficina, en, casa, muy, bu...","[[0.13769531, -0.023803711, 0.029663086, 0.197..."
249996,5,I have had Panasonic 3 phones for a long time....,1,"[i, have, had, panasonic, phones, for, long, t...","[[-0.22558594, -0.01953125, 0.09082031, 0.2373..."
249997,5,These are great pens to draw with and I like t...,1,"[these, are, great, pens, draw, with, i, like,...","[[-0.016235352, 0.09423828, 0.091796875, 0.119..."
249998,5,"I do martial arts, Jiu-Jitsu and I like to tak...",1,"[i, do, martial, arts, jiujitsu, i, like, take...","[[-0.22558594, -0.01953125, 0.09082031, 0.2373..."


In [40]:
import numpy as np
print(np.array(df["word_embeddings"].iloc[0]).shape)


(56, 300)


In [14]:
print(type(df["word_embeddings"].iloc[0]))

<class 'list'>


In [41]:
df


,star_rating,review_body,sentiment,cleaned_review,word_embeddings
169737,4,I got this exact calculator for middle school ...,1,"[i, got, this, exact, calculator, for, middle,...","[[-0.22558594, -0.01953125, 0.09082031, 0.2373..."
72272,2,"it's too solid, doesn't get in between the key...",2,"[its, too, solid, get, in, between, the, keys,...","[[0.01159668, 0.21679688, 0.14746094, 0.101074..."
208154,5,Perfect fit!!,1,"[perfect, fit]","[[0.03881836, -0.06640625, -0.09423828, -0.017..."
65426,2,bought these because they were pretty cheap an...,2,"[bought, these, because, they, were, pretty, c...","[[0.16699219, -0.05419922, -0.087402344, 0.019..."
30074,1,These batteries were not so good. I bought 5 o...,2,"[these, batteries, were, not, so, good, i, bou...","[[-0.016235352, 0.09423828, 0.091796875, 0.119..."
...,...,...,...,...,...
242776,5,Good product,1,"[good, product]","[[0.040527344, 0.0625, -0.017456055, 0.0786132..."
69906,2,It prints photosbut there are a lot of cons th...,2,"[it, prints, there, are, lot, cons, that, othe...","[[0.084472656, -0.0003528595, 0.053222656, 0.0..."
233708,5,Awesome Just What I needed for my printer work...,1,"[awesome, just, what, i, needed, for, my, prin...","[[0.07470703, -0.09033203, -0.055908203, 0.120..."
207131,5,I have had the blueparrot headssets forever an...,1,"[i, have, had, the, forever, the, dosent, last...","[[-0.22558594, -0.01953125, 0.09082031, 0.2373..."


In [ ]:
import numpy as np

max_review_length = 50  # Fixed review length
embedding_dim = 300  # Word2Vec vector size

def pad_or_truncate(embeddings, max_length=50, vector_size=300):
    if len(embeddings) == 0:
        return np.zeros((max_length, vector_size))  # Handle empty reviews properly

    if len(embeddings) >= max_length:
        return np.array(embeddings[:max_length])  # Truncate if too long

    # Pad if too short
    padding = np.zeros((max_length - len(embeddings), vector_size))
    return np.vstack((embeddings, padding))

# Apply padding/truncation
df["padded_embeddings"] = df["word_embeddings"].apply(lambda x: pad_or_truncate(x, max_review_length, embedding_dim))

# Convert to NumPy array
X = np.stack(df["padded_embeddings"].values)  # Shape: (num_samples, 50, 300)

# Save embeddings for training
#np.save("word_embeddings.npy", X)

In [42]:
df

,star_rating,review_body,sentiment,cleaned_review,word_embeddings
169737,4,I got this exact calculator for middle school ...,1,"[i, got, this, exact, calculator, for, middle,...","[[-0.22558594, -0.01953125, 0.09082031, 0.2373..."
72272,2,"it's too solid, doesn't get in between the key...",2,"[its, too, solid, get, in, between, the, keys,...","[[0.01159668, 0.21679688, 0.14746094, 0.101074..."
208154,5,Perfect fit!!,1,"[perfect, fit]","[[0.03881836, -0.06640625, -0.09423828, -0.017..."
65426,2,bought these because they were pretty cheap an...,2,"[bought, these, because, they, were, pretty, c...","[[0.16699219, -0.05419922, -0.087402344, 0.019..."
30074,1,These batteries were not so good. I bought 5 o...,2,"[these, batteries, were, not, so, good, i, bou...","[[-0.016235352, 0.09423828, 0.091796875, 0.119..."
...,...,...,...,...,...
242776,5,Good product,1,"[good, product]","[[0.040527344, 0.0625, -0.017456055, 0.0786132..."
69906,2,It prints photosbut there are a lot of cons th...,2,"[it, prints, there, are, lot, cons, that, othe...","[[0.084472656, -0.0003528595, 0.053222656, 0.0..."
233708,5,Awesome Just What I needed for my printer work...,1,"[awesome, just, what, i, needed, for, my, prin...","[[0.07470703, -0.09033203, -0.055908203, 0.120..."
207131,5,I have had the blueparrot headssets forever an...,1,"[i, have, had, the, forever, the, dosent, last...","[[-0.22558594, -0.01953125, 0.09082031, 0.2373..."


In [18]:
del word2vec_model

In [43]:
import numpy as np

max_review_length = 50  # Fixed review length
embedding_dim = 300  # Word2Vec vector size

def pad_or_truncate(embeddings, max_length=50, vector_size=300):
    if len(embeddings) == 0:
        return np.zeros((max_length, vector_size))  # Handle empty reviews properly

    if len(embeddings) >= max_length:
        return np.array(embeddings[:max_length])  # Truncate if too long

    # Pad if too short
    padding = np.zeros((max_length - len(embeddings), vector_size))
    return np.vstack((embeddings, padding))

# Apply padding/truncation
df["padded_embeddings"] = df["word_embeddings"].apply(lambda x: pad_or_truncate(x, max_review_length, embedding_dim))

# Convert to NumPy array
X = np.stack(df["padded_embeddings"].values)

In [44]:
import numpy as np
import torch
from torch import nn, optim
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Assuming df['padded_embeddings'] contains the padded word embeddings of size (50, 300)
# Let's also assume that df['label'] contains the binary sentiment labels (1 or 2)

# Define a custom Dataset class to handle large datasets more efficiently
class SentimentDataset(Dataset):
    def __init__(self, embeddings, labels):
        self.embeddings = embeddings
        self.labels = labels

    def __len__(self):
        return len(self.embeddings)

    def __getitem__(self, idx):
        embedding = torch.tensor(self.embeddings[idx], dtype=torch.float32)
        label = torch.tensor(self.labels[idx], dtype=torch.long)
        return embedding, label

# Convert to numpy arrays
X = np.stack(df['padded_embeddings'].values)  # Shape: (200000, 50, 300)
y = df['sentiment'].values - 1  # Subtract 1 to adjust labels to start from 0 (for PyTorch)

# Split the dataset into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create Dataset instances
train_dataset = SentimentDataset(X_train, y_train)
test_dataset = SentimentDataset(X_test, y_test)

# Create DataLoader for batching with smaller batch size
batch_size = 16  # Decrease the batch size to reduce memory usage
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

Using device: cuda


In [45]:
# Define the CNN Model
class SentimentCNN(nn.Module):
    def __init__(self, output_dim=2):
        super(SentimentCNN, self).__init__()

        # First Convolutional Layer
        self.conv1 = nn.Conv1d(in_channels=300, out_channels=50, kernel_size=3, padding=1)
        self.pool1 = nn.MaxPool1d(kernel_size=2)
        self.relu1 = nn.ReLU()

        # Second Convolutional Layer
        self.conv2 = nn.Conv1d(in_channels=50, out_channels=10, kernel_size=3, padding=1)
        self.pool2 = nn.MaxPool1d(kernel_size=2)
        self.relu2 = nn.ReLU()

        # Calculate the size of flattened features after the convolution layers
        self.flat_features = 10 * 12  # After two max pooling operations: 50 -> 25 -> 12

        # Fully Connected Layer
        self.fc = nn.Linear(self.flat_features, output_dim)

    def forward(self, x):
        # Input shape: (batch_size, seq_length, embedding_dim)
        # Transpose to: (batch_size, embedding_dim, seq_length)
        x = x.transpose(1, 2)

        # First conv layer
        x = self.conv1(x)
        x = self.pool1(x)
        x = self.relu1(x)

        # Second conv layer
        x = self.conv2(x)
        x = self.pool2(x)
        x = self.relu2(x)

        # Flatten
        x = x.view(-1, self.flat_features)

        # Fully connected layer
        x = self.fc(x)
        return x

In [46]:
# Train the CNN Model
def train_cnn(model, train_loader, test_loader, criterion, optimizer, epochs=10):
    model.to(device)  # Move model to GPU (if available)
    for epoch in range(epochs):
        model.train()
        total_loss = 0

        # Training loop
        for batch_idx, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)  # Move batch data to GPU
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        # Epoch evaluation
        model.eval()
        correct = 0
        total = 0

        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device)  # Move batch data to GPU
                outputs = model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        accuracy = correct / total
        print(f"Epoch {epoch+1}/{epochs} | Loss: {total_loss/len(train_loader):.4f} | Accuracy: {accuracy:.4f}")

    return accuracy

# Initialize the model
model = SentimentCNN(output_dim=2)

# Loss function (cross-entropy loss for binary classification)
criterion = nn.CrossEntropyLoss()

# Optimizer (Adam)
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model on GPU
num_epochs = 10
print("Training CNN Model...")
accuracy = train_cnn(model, train_loader, test_loader, criterion, optimizer, epochs=num_epochs)

print(f"Final Accuracy: {accuracy:.4f}")

Training CNN Model...
Epoch 1/10 | Loss: 0.3395 | Accuracy: 0.8775
Epoch 2/10 | Loss: 0.2704 | Accuracy: 0.8553
Epoch 3/10 | Loss: 0.2450 | Accuracy: 0.8851
Epoch 4/10 | Loss: 0.2259 | Accuracy: 0.8847
Epoch 5/10 | Loss: 0.2092 | Accuracy: 0.8826
Epoch 6/10 | Loss: 0.1943 | Accuracy: 0.8818
Epoch 7/10 | Loss: 0.1789 | Accuracy: 0.8799
Epoch 8/10 | Loss: 0.1670 | Accuracy: 0.8789
Epoch 9/10 | Loss: 0.1562 | Accuracy: 0.8781
Epoch 10/10 | Loss: 0.1442 | Accuracy: 0.8777
Final Accuracy: 0.8777


### Ternary Classification using CNN :

,star_rating,review_body
0,5,Great product.
1,5,What's to say about this commodity item except...
2,5,"Haven't used yet, but I am sure I will like it."
3,1,Although this was labeled as &#34;new&#34; the...
4,4,Gorgeous colors and easy to use
...,...,...
2640347,4,I can't live anymore whithout my Palm III. But...
2640348,4,Although the Palm Pilot is thin and compact it...
2640349,4,This book had a lot of great content without b...
2640350,5,I am teaching a course in Excel and am using t...


In [3]:
!ps aux --sort=-%mem | head -20

USER         PID %CPU %MEM    VSZ   RSS TTY      STAT START   TIME COMMAND
root         163 92.6  4.8 6382484 4281984 ?     Ssl  10:20   5:59 /usr/bin/python3 -m colab_kernel_
root         236  8.0  0.6 1299652 531888 ?      Sl   10:20   0:31 node /datalab/web/pyright/pyright
root         122 27.5  0.1 774752 123304 ?       Sl   10:20   1:47 /usr/bin/python3 /usr/local/bin/j
root           7 10.7  0.1 934504 96860 ?        Sl   10:20   0:42 /tools/node/bin/node /datalab/web
root          81  0.0  0.0  63632 50928 ?        S    10:20   0:00 python3 /usr/local/bin/colab-file
root         221  0.0  0.0 1243316 21380 ?       Sl   10:20   0:00 /usr/colab/bin/language_service -
root         208  0.3  0.0 545964 20876 ?        Sl   10:20   0:01 /usr/bin/python3 /usr/local/lib/p
root          12  9.8  0.0 1239892 17648 ?       Sl   10:20   0:38 /usr/colab/bin/kernel_manager_pro
root         123  0.1  0.0 1231596 9140 ?        Sl   10:20   0:00 /usr/local/bin/dap_multiplexer --
root           9

In [6]:
import gc, torch
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()

In [4]:
!ps -o ppid= -p 80


      7


In [ ]:
!kill -9 7
